<h2>Exploratory Data Analysis (#1)</h2>

In [ ]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load data

data = pd.read_csv("data//TrainData.csv") # This file is an inner join on the order name of the menu and the training data

In [ ]:
# Basic data lookup

data.info()
data.nunique()
# data.isnull().sum() # No missing values
# data.head()

In [ ]:
# Modify features, a lot of this is just making it easier to read for me

# data = data.drop(['Order'], axis=1) # Assume the order name doesn't provide meaningful value, since order number exists
data = data.drop(['Number'], axis=1) # Assume the order number doesn't provide meaningful value, since order is categorical
# data['Number'] = data.Number.astype(object) # If number isn't being dropped due to redundancy, it should be categorical not numerical
data['Year'] = data.Year.str.split().str.get(1) # The actual year number matters more than the word "year"
data['University'] = data.University.str.replace("University", "") # Get rid of the word "university"
data.info()
data.head()

In [ ]:
# Clean data

data['Major'] = data.Major.str.replace(" ", "")
data['University'] = data.University.str.replace(" ", "")
# data['Year'] = data.Year.astype(int) # Year is a category, not a number
data.info()
data.head()

In [ ]:
# View data values again

print(data.Major.unique())

In [ ]:
# Some more data cleaning, a lot of this is just making it easier to read for me

data['University'] = data.University.str.replace("Indiana-PurdueIndianapolis(IUPUI)", "IUPUI", regex=False)
data['University'] = data.University.str.replace("ofNotreDame", "NotreDame")
data['University'] = data.University.str.replace("ofEvansville", "Evansville")
print(data.University.unique())
data.info()
data.head(20)

In [ ]:
# Take a look at the data spread, preliminary statistics

# data.describe().T
data.describe(include='all').T

In [ ]:
# Separate data into numerical and non-numerical values

cat_cols = data.select_dtypes(include=['object']).columns
num_cols = data.select_dtypes(include=[np.number]).columns

In [ ]:
# Visual skew and pattern of numerical data

for col in num_cols:
    print(col)
    print('skew', data[col].skew(), sep=':')
    print('mean', data[col].mean(), sep=':')
    print('mode', data[col].mode(), sep=':')
    print('variance', data[col].var(), sep=':')
    plt.figure(figsize=(15,4))
    plt.subplot(1,2,1)
    data[col].hist(grid=False)
    plt.ylabel('count')
    plt.subplot(1,2,2)
    sns.boxplot(x=data[col])
    plt.show()

In [ ]:
# Visual categorization and pattern of categorical data

fig, axes = plt.subplots(3,1, figsize=(18,18))
sns.countplot(ax=axes[0], x='Major', data=data, order=data['Major'].value_counts().index)
sns.countplot(ax=axes[1], x='University', data=data, order=data['University'].value_counts().index)
sns.countplot(ax=axes[2], x='Order', data=data, order=data['Order'].value_counts().index)

<h4>Univariate Analysis Findings</h4>

The above univariate analysis graphs show that there aren't any particular variables or features within this data that are severely skewed one way or another. 

Some interesting features include the bimodal distibution of the price, and the lack of data on 1st years and 4th years.

The university and major count data are fairly biased towards certain categories, such as Butler University and Chemistry. It is unlikely that this data truly reflects the population percentages, because according to the information given, this data was collected on a voluntary basis, i.e., students chose to go up to the food trucks to submit their orders instead of randomly or categorically being selected to do so. This implicates several biases within the population data, so random sampling will need to be done in order to eliminate said bias.

Due to the incredibly tiny subpopulations of certain university categories or year categories, it is difficult to truly have a categorical random sample to reduce the data bias. This will be taken into consideration during the model selection process.

In [ ]:
# Comparing numerical variables against each other

plt.figure()
sns.pairplot(data=data)
plt.show()

In [ ]:
# Comparing categorical variables against the output numerical variable

data.groupby(['University','Order']).size().unstack().plot(kind='bar',stacked=True, figsize=(18,12)) 
data.groupby(['Major','Order']).size().unstack().plot(kind='bar',stacked=True, figsize=(18,12)) 

<h4>Bivariate Analysis Findings</h4>

The above bivariate analysis graphs show that the university and major categories do seem to vary the order proportions. However, with this many categories for each variable compared to each other (10 orders versus 10 universities or 20 majors), it is almost impossible to form any sort of analytical, numerically-backed conclusion from visual inspection.

The different stackings, though, show that there is some variation between universities and majors on the orders submitted.

In [ ]:
# Generate heat map

plt.figure()
sns.heatmap(data.corr(), annot=True, vmin=-1, vmax=1)
plt.show()

<h4>Multivariate Analysis Findings</h4>

The above heat map of numerical values shows something that was hinted by the univariate and bivariate graphs: there is little correlation between features. 

The highest correlation between two features is price vs calories, with a correlation value of 0.75, which is fairly strong. This is backed up by the corresponding pair plot shown in the bivariate section. 

All other correlations have a value between -0.25 and 0.05. This indicates that all of these are fairly weak correlations. Assuming that all of these features have an impact on the final output / categorization, they do not overlap so they will all contribute "new" data from other variables. That is, it is difficult to derive the value of these variables from other variables in the heat map.

<h4>Impactful Business Use Cases</h4>

This data can see several more potentially-impactful use cases in the business:

1. Determining which types of food are more commonly ordered from food trucks depending on universities or locales. Even though this does not indicate a specific order for each person, it still provides statistics on which orders are more common and where. These data can be used to match demand with supply, sending trucks to locations where their food is more frequently ordered. This is a good business use for the data in the food truck industry.
2. Determining which types of food are more commonly ordered at which times. This correlation provides information on when each type of food is more commonly ordered, and therefore can help the food truck workers decide what more of what type of food to prepare depending on the hour. This can help prevent food waste and therefore save food trucks in money / business costs.
3. Any sort of correlation between majors and type / time of food ordered can help the food trucks determine where to locate themselves on universities. If a certain type of major is more likely to order a certain type of food, then it is good business practice for a food truck that provides such food to locate itself near that major's department building on university grounds. In addition, if the food truck plans on moving around the university, it can choose to move to be near different major department buildings based on the hour, given that different majors order at different hours. 
4. Assuming that different years have different dorm or living locations, and that they all live nearby each other, food trucks can use time of order as well as type of order correlated with student years to determine which years to cater more towards as well as which dorm buildings to be located nearest to.
5. In terms of general marketing that doesn't have anything to do with food trucks, it can be assumed that certain majors have certain material or shopping requirements, such as chemistry majors needing lab coats or mathematics majors needing mathematical calculation software. Due to an occurence of more of some types of majors existing at some universities, vendors who sell these products can focus their marketing on those universities in order to have a higher marketing yield, and target certain majors of students to sell those products.

All of these data can be used in the above ways to benefit businesses. This indicates that data can be used in multiple ways, so each piece of data collected can yield a lot more information than expected.

<h2>Implications of Data (#2)</h2>

<h4>Ethics Regarding Data</h4>

This data is collected from a student explicitly for one purpose, and that is to guess at the student's order before the student even places it. This is most likely explained very clearly to the student in the interactive promotion, because that is the whole point of the interactive promotion from the student perspective. Given unrelated data about the student, the student's order is guessed.

It should be noted that the student doesn't explicitly know how the order will be guessed: a person, probably the chef, could guess the order, which is what FoodX has been doing thus far; alternatively, an AI algorithm could guess the order, which is what this development is all about. In general, a student should know where their information is going, how it is used, and with what other information it is connected. 

For instance, if a human chef is using the information to guess at a student's order, the information should not go anywhere else nor be stored for further use, because its purpose has been accomplished. However, if an artificial intelligence algorithm is using the data, then the data must necessarily be stored in a system. This can allow the data to be used later, for purposes that the student isn't aware of. This can easily lead to exploitation of the data and of the student themself. In effect, using and storing this data should not violate a student's privacy. To that end, all data and where it is being used, and what it is being fed to should be transparent to the student when they hand it over. This is especially true for AI algorithms, which can draw conclusions about a student that would otherwise not be possible. If a student doesn't want to come under such scrutiny by a private enterprise, then it is ethically within their right to refuse it. While the provided training data is anonymized, such data while tied to purchase history and payment information can easily prove a breach of privacy. If the food trucks accept credit card payment, or keep track of purchases at all, it will not be difficult to de-anonymize the dataset, at least to an extent.

It should also be noted that data is often implicitly biased. If for example a university population has for a higher population of certain demographics and minority groups, or the data is collected at a location that has a higher population of certain demographics and minority groups, then the data can produce unfavorable biases or make assumptions about those demographics. This becomes especially dangerous if that assumption propagates and is used in future models for advertisement or other use.

Further, in order to ensure that the data is used properly and isn't targeting towards individuals, all data used should be up to date. Using old data can lead to the data appearing misleading or simply untrue. Data presented by AI algorithms tends to confirm already-present biases and gravitate towards the average. In case of requiring more diversity, AI algorithms aren't the proper tools and so should not be used. In addition, due to the black box nature of many of many of these algorithms, their results should not be presented without explicit uncertainty.

Finally, whether or not the AI algorithm is deterministic should also matter. Given the relatively few parameters provided, it is very possible to have the exact same scenario occur multiple times but with different results. Many AI classifiers use probabilistic models instead of deterministic, so the level of determinism in play should be remembered and accounted for. Just because an AI model claims something as probable does not indicate that it is or that it will occur.

<h4>Business Outcomes</h4>

Data about a student or a population can lead to insights that are useful in many different scenarios. Some of these scenarios have been highlighted earlier, and include focused marketing schemes based on types of food and matching food truck supply with demand across university campuses. However, the biases implicit within the collected data can paint an incorrect picture of what is actually transpiring. Moving locations based on a correlation of food and major may not pan out to be correct, because perhaps only a small fraction of a major's population present in an area prefers a type of food. Perhaps the rest of the population will not enjoy the same food due to subpopulation segmentation. Such biases should be kept in mind, and assumptions regarding the data should all be annotated, because assumptions during data collection can easily lead to falsely drawn conclusions. Because these data are used to guide important business decisions, their biases and metadata should be widely understood.

Data collection, done properly, also presents a lucrative business venture. These data can be used in a vast area by many different types of companies. There are some legal limits to data collection. However, within the FoodX business itself, data can be used to simply understand the customers better. What demographics does FoodX serve now? What demographics will FoodX need to expand to for greater business opportunity? What do customers care about from FoodX? What food options do they prefer and why? Where are the optimal locations to posThese data can answer all of the above questions and more. Answering these questions will allow FoodX to better cater to their customers, gain more customers, as well as gain valuable insight into the business model to see what works and what doesn't. 

Data storage must be secure. Big data collected over populations can take a vast amount of memory, and due to its demand is vulnerable to cyberattack. Cyberattack can reflect badly on business, as well as cause a loss of trust for stakeholders and customers. Therefore, security of stored data is paramount. In addition, it is very easy for stored data to go out of date simply through the passage of time. However, old data can yield drastically wrong conclusions. It is paramount to update stored data as often as possible so it matches up as closely with reality.

<h4>Technical Factors</h4>

Data collection can significantly impact the quality of the training set of the algorithm. If the data is collected using proper sampling techniques, or if the biases within the sample are addressed and accounted for such as through categorical random selection, then the training set itself tends to eliminate many problems that can otherwise crop up. These problems include algorithm underfitting, specificity to a particular category, as well as a tendency towards the bias. AI algorithms generally tend towards implicit biases within the dataset, so it is imperative to eliminate as many of these biases as possible. 

Unfortunately, no data set is completely unbiased, so corrections must be made to reduce this bias. In the case of AI algorithms, data bias stems from data collection. In this case, the data cannot be collected in an unbiased manner due to the voluntary nature of the food truck promotion as well as the nature of the demand for food. Therefore, this self-selection bias, as well as the population bias inherent to selling food on various university campuses, must be corrected for.

Other factors to consider involve the data storage techniques and structures. If the dataset is large enough, then drawing data from storage can cause the model to take an excessive amount of time to train. Certain databases and structures, such as an hdf5 structure, can shorten the amount of time to draw data from memory. Whether the data is stored in table format, how clean it is, whether there are any missing values, or other such pre-use data modification and wrangling can also have an impact on which data is used for which purpose. A lot of missing values in a particular dataset can indicate a deeper problem with collection techniques or the data staying up-to-date. In this sense viewing the data itself has importance in the model's performance, and whether it can operate on a broader dataset.

<h2>Model (#3)</h2>

<h4>Process Outline</h4>

1. A model needs to be selected based on the needs of the project.
2. The data needs to be cleaned and prepared for training. The data needs to be converted to vector format.
3. The data is split into a training and testing set, and the training set is passed to fit the model.
4. The trained model is pickled and saved.
5. The testing set is used to determine the trained model's accuracy.

The dataset used for training the model, TrainData.csv, is a join on the Xtern_TrainData page and the Menu page of the XTern 2024 Artificial Intelegence Data Set file provided.

In [139]:
# Import libraries

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

import pickle
import pandas as pd
import numpy as np

<h4>Model Selection</h4>

In order to select a good model for this dataset, we need to look at the following factors:
1. We have about 5000 lines of data.
2. The data varies between continuous numerical data and categorical data. 
3. The output is categorical, a classification between one of ten different states.
4. Storage and speed aren't a primary consideration, but are a limitation due to the hardware restrictions I am working with.

Logistic regression is a good model baseline. However it is often used in binary classification; here, we have many more classes to choose from. Due to the simplicity of the model, it doesn't perform as well when faced with more complex classification problems. Naive Bayes models, while simple to implement, tend high towards bias. Because we do not have a high enough subpopulation size for certain subpopulations, such as year 1 and year 4 students, or certain universities like DePauw or Indiana University Bloomington, it is difficult to have random sampling of subpopulations to reduce sample bias. A k-nearest model might provide a good result. However, k-nearest models tend to scale in memory use by the size of the complete data set. Due to the relatively large size of n=5000 values, k-nearest isn't the first choice. A classification decision tree is a relatively fast model that can handle complex classification categories. However, due to the structure of the decision tree, it tends to overfit. 

Based on all the above parameters, we will use a classification decision tree model and compare its performance to a logistic regression model. 

In order to make up for the overfitting of a decision tree, we can also use a random forest classifier to search for the best parameters to use in the decision tree. While we have not done in the below model, a random forest classifier will theoretically produce the best results for this type of classification problem.

In [140]:
# Define the model

model = KNeighborsClassifier()

<h4>Data Preparation</h4>

To prepare the data to be loaded into a model, it needs to be converted into vector features. A simple way to convert a model into vector features is to simply convert all features into numerical representations and then plot them as a vector, that is, label encoding. This is a simple, rather naive way to convert features into vectors. A second method we can use involves turning each class in a categorical feature its own feature, and labeling it as 1 or 0 to determine if it is that class or not. We will be using both methods and then determining which method works best, per model.

More complicated methods involve pre-weighting certain features, or using NLP-type embeddings to view all the features as a whole and convert them into a total vector. 

Another aspect to consider, for this model, is the additional data tied to each conclusion category or order decision. Each order has an attached number of calories and a price. We can try to take this additional data into account by training different models to predict either order number, order price, or order calories. Whichever model has the best fit is chosen. If we want to go further, we can use all three models, and then average the results for our final order prediction. (We did find earlier that price and calories are highly correlated. It is unlikely that using both will be helpful in many cases.)

In [141]:
# Load data

data = pd.read_csv("data//TrainData.csv")

In [142]:
# Process data for label encoding

data = data.drop(['Order'], axis=1) # We need to make all data numerical, and convert object data into numerical data to create feature vectors

data['Year'] = data.Year.str.split(" ").str.get(1) 
data['Year'] = data.Year.astype(int) 

major_replacement = dict(zip(data.Major.unique(),[i+1 for i in range(20)]))
data['Major'] = data.Major.replace(major_replacement)
data['Major'] = data.Major.astype(int)

university_replacement = dict(zip(data.University.unique(),[i+1 for i in range(10)]))
data['University'] = data.University.replace(university_replacement)
data['University'] = data.University.astype(int)

In [143]:
# Convert label-encoded data to feature vectors

x = np.asarray([np.asarray(data['Year']),np.asarray(data['Major']),np.asarray(data['University']),np.asarray(data['Time'])]) # We have 4 independent variables
y = np.asarray([np.asarray(data['Number']),np.asarray(data['Price']),np.asarray(data['Calories'])]) # We have 3 dependent variables

y = y[0] # To predict only the order number [0], to predict the price [1], to predict the calories [2]
x = x.transpose() # To match y.shape
y = y.reshape(-1,1)

In [144]:
# Process labeled data for one-hot encoding, optional

ct = ColumnTransformer(transformers=[('x',OneHotEncoder(),[0,1,2,3])], remainder='passthrough',)
ct.fit_transform(x)
ct = ColumnTransformer(transformers=[('y',OneHotEncoder(),[0])], remainder='passthrough',)
ct.fit_transform(y)

<5000x10 sparse matrix of type '<class 'numpy.float64'>'
	with 5000 stored elements in Compressed Sparse Row format>

<h4>Model Training</h4>

Splitting the dataset into training and testing, in this case, can be randomly determined. There are no known ties between any of the data values. For example, we don't know and can't tell if two of the values come from the same person. If two values were to come from the same person, we would need to put both values in either the training or the testing set; the data set must be split at the person level, not the individual value level. Otherwise, the testing is invalidated, as the testing set should be completely separate from the training set.

All models should be trained and tested using the same data. 

In [145]:
# Split data into training and testing groups

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=21)

In [146]:
# Search for best parameters for random forest model, to go into training

# def best_params():
#     classification_params = {
#         "n_estimators": [10, 50, 100],
#         "max_features": ["auto", "sqrt", "log2"],
#         "max_depth": [5, 10, 20, 50],
#         "min_samples_split": [2, 4, 6],
#         "min_samples_leaf": [2, 4, 6],
#         "bootstrap": [True, False],
#     }

#     classifier = RandomForestClassifier(random_state=21)

#     random_search = RandomizedSearchCV(
#         estimator=classifier,
#         param_distributions=classification_params,
#         n_iter=10,
#         cv=2,
#         # verbose=2,
#         random_state=21
#     )

#     random_search.fit(x_train,y_train)
#     return random_search.best_params_

# if model.__class__ == RandomForestClassifier:
#     kwparams = best_params()
#     model = RandomForestClassifier(**kwparams, random_state=21)

In [192]:
# Define the number of neighbors for k-neighbors classification

if model.__class__ == KNeighborsClassifier:
    n = 6
    model = KNeighborsClassifier(n_neighbors=n)

In [193]:
# Train model on training data

model.fit(x_train, y_train)

/Users/siddharth_khadkikar/Library/CloudStorage/OneDrive-Personal/Rando/Code/Python Stuff/Xtern/xtern_venv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=6)

In [194]:
# Pickle trained model

model_name = "nearest_neighbors_model"
pickle.dump(model, open("pickled_models//"+model_name,'wb'))

<h4>Model Testing</h4>

Each model is tested by submitting the testing data set. Then, the predicted orders are compared to the actual orders. The accuracy score represents the proportion of classifications the model correctly predicted. The classification report displays additional data on the model, such as the number of instances of each class, the F-score per class, the true positive rate per clsas, and the precision per class.

In [195]:
# Depickle model

file_name = "pickled_models"
model_name = "nearest_neighbors_model"
model = pickle.load(open(file_name+"//"+model_name,'rb'))

In [196]:
# Test model on testing date

predicted_orders = model.predict(x_test)

accuracy = accuracy_score(predicted_orders, y_test)
report = classification_report(predicted_orders, y_test)

print("accuracy\n"+str(accuracy), report, sep='\n')

accuracy
0.608
              precision    recall  f1-score   support

           0       0.67      0.68      0.68       100
           1       0.62      0.48      0.54       132
           2       0.52      0.57      0.55        89
           3       0.87      0.69      0.77       117
           4       0.58      0.62      0.60        95
           5       0.69      0.59      0.64       105
           6       0.42      0.63      0.50        71
           7       0.73      0.67      0.70       108
           8       0.47      0.61      0.53        77
           9       0.55      0.57      0.56       106

    accuracy                           0.61      1000
   macro avg       0.61      0.61      0.61      1000
weighted avg       0.63      0.61      0.61      1000



<h4>Results</h4>

The decision tree classifier has the highest accuracy rating of around 0.65, compared to the other tested models. In all, the decision tree model and the random forest model ended up with very similar accuracy ratings, with the random forest not improving much upon the decision tree's performance. The logistic regression model ended up with the lowest accuracy rating of around 0.25, depending on the size of the training set. The support vector model ended up with an accuracy rating of around 0.42. The K-nearest neighbors model ended up having an accuracy rating of 0.61 when its k-value was optimized at at around k=6.

Generally, the larger the proportion of the training set, the higher the model's performance. However, increasing the training test size by 1000 only increased the accuracy rating by a couple of percent, so the yield isn't high.

<h2>Considerations (#4)</h2>

Given the work required to bring this solution to maturity, the following considerations must be taken into account.

1. The profit loss due to the discount. At this point, the best model supposedly succeeds only about 60% of the time. This means that 40% of the customers will have a 10% discount. This discount will decrease profits. 
2. The loss of profit can be balanced by the increase in customer attraction through marketing. If the amount of new customers that this interactive promotion brings in boosts profit so that the loss incurred by the 10% discount is more than made up, then this promotion could be a suitable course of action. If the new profits merely break even with the loss, then using this promotion and the AI prediction simply becomes a waste of resources and time.
3. The data management task of keeping the customer data updated, server and security management for storing and protecting customer data, as well as the time cost of feeding additional data to the algorithm and waiting for confirmation on whether the algorithm guessed correctly or not, all make the cost of the promotion much higher than simply the 10% discount.
4. If this data could be ethically used for future data business ventures, such as choosing better setup locations for food trucks, determining which food trucks better match food demand, and gaining insight to customer preferences and personalities, then the promotion's ultimate yield can end up higher than the simple increase of customers. The data collected could also be used to help the customers become more satisfied with FoodX.
5. The data privacy, security, and ethics risks that come with using AI in such a manner must be dealt with before this promotion can be seriously considered. Customer data, especially sensitive data, should not be collected unless FoodX has the access to protect said data. Finally, FoodX must be careful to not violate customer privacy bounds and stay within ethical limits while using this data.